In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

##############################################################
import json
import re

# ✅ JSON 파일 읽기
file_path = "/content/drive/MyDrive/busanit501-1234/트럼프_naver_news.json"

with open(file_path, "r", encoding="utf-8") as file:
    news_data = json.load(file)  # JSON 데이터 로드
##############################################################

# ✅ 샘플 데이터셋 (문장 예제)
corpus = [
   "나는 너를 사랑해",
    "나는 코딩을 좋아해",
    "너는 나를 좋아해",
    "너는 파이썬을 공부해",
    "우리는 인공지능을 연구해",
    "딥러닝은 재미있어",
    "파이썬은 강력해",
    "나는 자연어처리를 공부해",
]
##############################################################
# ✅ JSON 데이터에서 'title' 값만 추출하고 한글만 남기기
def extract_korean(text):
    """문장에서 한글만 남기는 함수"""
    return re.sub(r"[^ㄱ-ㅎ가-힣 ]+", " ", text)

news_titles = [extract_korean(item["title"]) for item in news_data if "title" in item]

# ✅ corpus에 한글만 남긴 뉴스 제목 추가
corpus.extend(news_titles)

# ✅ 결과 출력
print("📌 최종 corpus 리스트:")
print(corpus)
##############################################################

# ✅ 단어 사전 만들기 (Tokenization)
word_list = list(set(" ".join(corpus).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
idx_dict = {i: w for w, i in word_dict.items()}

📌 최종 corpus 리스트:
['나는 너를 사랑해', '나는 코딩을 좋아해', '너는 나를 좋아해', '너는 파이썬을 공부해', '우리는 인공지능을 연구해', '딥러닝은 재미있어', '파이썬은 강력해', '나는 자연어처리를 공부해', ' 싸운 건  쇼 로이터   미 우크라 간 광물협정 체결할 수도 ', ' 속보 트럼프   무역전쟁 에도   올해 경제성장률   안팎  목표', '뉴욕증시  관세전쟁 격화에  일 연속 하락 마감', ' 우크라 광물협정 임박 보도에 재무  지금은 아냐 ', ' 트럼프  개입했지만  투어 커미셔너  와 통합 쉽지 않다 ', '  엔비디아 급반등 이어 삼성전자    하이닉스  상승 출발 ', '미국 부통령  우크라 파병 추진 유럽국에  어중이떠중이  지칭 논란', '루비오 방문 한 달만에 홍콩 기업 결국 파나마운하 항만지분 미국 기업 ', ' 트럼프  관세   협상용 엄포  아닌  진심 그럼에도   증시 기 ', '전진건설로봇 주가 방긋 우크라이나 전쟁 종전 임박', ' 물동량  만 시대 마중물  인천신항 완전자동화부두 조성 순항', ' 해외브리핑  미    억  규모 태양광 프로그램 보조금 동결 해제', '코스피    증시 변동성 속 상승 출발 선 등락', ' 미국 술 치워라 캐나다 주 정부들  트럼프  관세에 맞불', '  김여정  칼빈슨함  발끈하자  한국 방어 공약 분명 ', '대성하이텍  방산 수주 비중 급증 유럽 및 이스라엘과 장기 계약 기대 ', '코스닥  외인 기관 사자에  대 상승 선', '미 상무  중간서 만날 수도 캐 멕 관세 경감 가능 시사', ' 트럼프  캐나다  총리 를  주지사 로 호칭 상호관세 부과 으름장', ' 상무  트럼프   멕 관세 경감 고려 내일 발표할 듯 상보 ', '  원조중단에 속탄 젤렌스키  백악관 일 유감 바로잡을 때 ', '  국방차관 후보  한국 정치상황에 공개 우려 입장', '예수회난민봉사기구  지원 중단에 활동 위축 우려', ' 보스레이디 로 변신한 멜라니아 슈트 빼입고 성큼 돌진'

In [4]:
# ✅ 데이터셋 변환
def make_data(corpus):
    inputs, targets = [], []
    for sentence in corpus:
        words = sentence.split()
        for i in range(len(words) - 1):  # "I love" -> "you"
            x = [word_dict[w] for w in words[:i+1]]
            y = word_dict[words[i+1]]
            inputs.append(x)
            targets.append(y)

    return inputs, targets

inputs, targets = make_data(corpus)

# ✅ 패딩 추가 (문장 길이를 맞춤)
max_len = max(len(seq) for seq in inputs)
inputs_padded = [seq + [0] * (max_len - len(seq)) for seq in inputs]
targets = torch.tensor(targets, dtype=torch.long)

# ✅ 데이터셋 및 DataLoader 생성
class TextDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long)
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

dataset = TextDataset(inputs_padded, targets)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

vocab_size = len(word_dict)  # 단어 개수
embed_size = 10  # 임베딩 차원
hidden_size = 16  # RNN 은닉층 크기
num_classes = len(word_dict)  # 예측할 단어 개수

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

# ✅ GRU 모델 정의
class GRUTextModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(GRUTextModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # 단어 임베딩
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  # 마지막 시점의 GRU 출력을 사용
        return out

# ✅ 모델 생성
model = GRUTextModel(vocab_size, embed_size, hidden_size, num_classes)

# ✅ GPU 사용 가능하면 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:
num_epochs = 100
print("🚀 GRU 모델 학습 시작...")
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# ✅ 모델 저장
model_path = "/content/drive/MyDrive/busanit501-1234/gru_news_model.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ 학습된 모델이 저장되었습니다: {model_path}")


🚀 GRU 모델 학습 시작...


In [ ]:
model_path = "/content/drive/MyDrive/busanit501-1234/gru_news_model.pth"
# ✅ 저장된 RNN 모델 불러오기
def load_model(model_path, vocab_size, embed_size, hidden_size, num_classes):
    model = LSTMTextModel(vocab_size, embed_size, hidden_size, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model.eval()
    return model

# ✅ 모델 불러오기
loaded_model = load_model(model_path, vocab_size, embed_size, hidden_size, num_classes)
print("✅ 모델이 성공적으로 불러와졌습니다!")


✅ 모델이 성공적으로 불러와졌습니다!


In [ ]:

import torch.nn.functional as F

def predict_next_word(model, sentence):
    """
    저장된 RNN 모델을 사용하여 주어진 문장의 다음 단어를 예측하는 함수.
    """
    # ✅ 입력 문장을 정수 인코딩
    words = sentence.split()
    input_seq = [word_dict[w] for w in words if w in word_dict]

    # ✅ 패딩 추가 (길이를 맞추기 위해)
    input_padded = input_seq + [0] * (max_len - len(input_seq))
    input_tensor = torch.tensor([input_padded], dtype=torch.long)

    # ✅ 모델 예측
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_idx = torch.argmax(probabilities).item()
        confidence = probabilities[predicted_idx].item()

    predicted_word = idx_dict[predicted_idx]

    print(f"🔍 입력 문장: '{sentence}'")
    print(f"📊 예측된 단어: '{predicted_word}'")
    print(f"✅ 예측 확률: {confidence * 100:.2f}%")

# 🏆 샘플 문장 예측 실행
sample_sentence = "호칭  "
predict_next_word(loaded_model, sample_sentence)

🔍 입력 문장: '호칭  '
📊 예측된 단어: '트럼프'
✅ 예측 확률: 3.18%
